In [8]:
from jupyter_dash import JupyterDash

In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, ALL, MATCH
from dash.exceptions import PreventUpdate
import json

import dash_bootstrap_components as dbc
from datetime import datetime as dt
import pandas as pd
import numpy as np
import time
import random
import plotly.express as px
import plotly.graph_objs as go
import re
import locale
import datetime

In [43]:
# df_art = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_deduplicated_v4.json')
# df_articles = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_tag_clean.json')
# df_art = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_tag_clean_vf.json')
df_articles = pd.read_json('/home/sid2018-6/projetInterPromo2021/df_clean_tagNA.json')
df_news = pd.read_json('/home/sid2018-6/projetInterPromo2021/Pred_Global_V0.json')
# df_resume = pd.read_json('/home/sid2018-6/projetInterPromo2021/summary_v0.json')
df_resume = pd.read_json('/home/sid2018-6/projetInterPromo2021/summary_v1.json')
df_gamme_ges = pd.read_json('/home/sid2018-6/projetInterPromo2021/Res_gestion_semi_supervised_V1.json')

In [44]:
def date_propre(extract_datetime):
    chaine = str(extract_datetime)
    mois_fr = ['janvier','février','mars','avril','mai','juin','juillet','août',
               'septembre','octobre','novembre','décembre']
    m_fr = ['janv','fév','mars','avril','mai','juin','juil','aout','sept','oct','nov','déc']
    c1 = re.compile(r'\d{2}/\d{2}/\d{2}')
    c2 = re.compile(r'\d+ \w+ \d{4}')
    c3 = re.compile(r'\w+ \d+, \d{4}')
    c4 = re.compile(r'\w+  \d{4}')
    c5 = re.compile(r'\d+.\d+.\d+')
    c6 = re.compile(r'\w+.\d+')
    
    
    if c1.match(chaine) is not None:
        chaine = c1.findall(chaine)[0]
        if len(chaine.split('/')[2]) == 2:
            return dt.strptime(chaine, '%d/%m/%y').strftime('%d/%m/%Y')
        else :
            return dt.strptime(chaine, '%d/%m/%Y').strftime('%d/%m/%Y')
    
    elif c2.match(chaine) is not None:
        try:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        except:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%d %B %Y').strftime('%d/%m/%Y'))
        
    elif c3.match(chaine) is not None:
        chaine = chaine.split()
        if len(chaine[1])==2:
            chaine[1] = '0'+str(chaine[1])
        if chaine[0].lower() in mois_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) == 4 or 'é' in chaine[0]:
            chaine[0]=chaine[0]+'.'
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
        elif len(chaine[0]) >=5:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(' '.join(chaine), '%B %d, %Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(' '.join(chaine), '%b %d, %Y').strftime('%d/%m/%Y'))
    
    elif c4.match(chaine) is not None:
        locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
        return(dt.strptime(chaine, '%B %Y').strftime('%d/%m/%Y'))
    
    elif c5.match(chaine) is not None:
        chaine = chaine.strip()
        if len(chaine.split('.'))!=1:
            if len(chaine.split('.')[2]) == 4:
                return(dt.strptime(chaine, '%d.%m.%Y').strftime('%d/%m/%Y'))
            else:
                return(dt.strptime(chaine, '%d.%m.%y').strftime('%d/%m/%Y'))
        else:
            return time.strftime("%d/%m/%Y", time.localtime(int(int(chaine) / 1000)))
    
    elif c6.match(chaine) is not None:
        chaine = chaine.strip()
        if chaine.split('.')[0].lower() in m_fr:
            locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
            return(dt.strptime(chaine, '%b%Y').strftime('%d/%m/%Y'))
        else:
            locale.setlocale(locale.LC_ALL, 'en_GB.UTF-8')
            return(dt.strptime(chaine, '%b.%Y').strftime('%d/%m/%Y'))
    
    else:
        if c2.findall(chaine)!=[]:
            return date_propre(c2.findall(chaine)[0])
        elif c1.findall(chaine)!=[]:
            return date_propre(c1.findall(chaine)[0])
        elif c3.findall(chaine)!=[]:
            return date_propre(c3.findall(chaine)[0])
        else:
            return None
    


In [45]:
# df_articles['art_title'] = df_articles['art_title'].replace('', np.nan, inplace=True)
# df_articles = df_articles.dropna(subset=['art_title', 'art_url','art_content'])
# df_articles[df_articles['art_title']==' '].index.values.astype(int)
# df[df['A']==5].index.values.astype(int)
df_articles

,art_auth,art_content,art_content_html,art_extract_datetime,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_type,src_url
0,None,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,1,fr,None,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,http://fncdg.com/wp-content/uploads/2020/09/im...,FNCDG,xpath_source,http://fncdg.com/actualites/
1,None,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,2,fr,None,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,http://fncdg.com/wp-content/uploads/2020/03/co...,FNCDG,xpath_source,http://fncdg.com/actualites/
2,None,Quels étaient les objectifs poursuivis par le ...,"<p style=""text-align: justify;""><strong>Quels ...",24 octobre 2019,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,xpath_source,http://fncdg.com/actualites/
3,None,"La journée thématique, qui aura lieu durant le...","<p style=""text-align: justify;""><strong>La jo...",31 mai 2017,27,fr,None,Journée Thématique FNCDG « Les services de san...,http://fncdg.com/journee-thematique-fncdg-les-...,http://fncdg.com/wp-content/uploads/2017/05/pu...,FNCDG,xpath_source,http://fncdg.com/actualites/
4,None,La 1ère journée thématique en région sur le th...,"<p style=""text-align: justify;"">La 1<sup>ère</...",13 mars 2017,28,fr,None,Journée Thématique FNCDG « Vers de nouveaux mo...,http://fncdg.com/journee-thematique-fncdg-vers...,http://fncdg.com/wp-content/uploads/2017/03/Sa...,FNCDG,xpath_source,http://fncdg.com/actualites/
5,None,L’une des innovations de la loi n°2019-828 du ...,"<p style=""text-align: justify;"">L’une des inno...",22 octobre 2020,30,fr,None,La publication d’un guide d’accompagnement à l...,http://fncdg.com/la-publication-dun-guide-dacc...,http://fncdg.com/wp-content/uploads/2020/10/LG...,FNCDG,xpath_source,http://fncdg.com/actualites/
6,None,"La FNCDG mène, en collaboration avec d’autres ...","<p style=""text-align: justify;"">La FNCDG mène,...",10 décembre 2020,31,fr,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,http://fncdg.com/wp-content/uploads/2020/12/im...,FNCDG,xpath_source,http://fncdg.com/actualites/
7,None,"Créé pour et par les décideurs territoriaux, É...","<p style=""text-align: justify;"">Créé pour et p...",24 février 2017,32,fr,None,Lancement du réseau Étoile,http://fncdg.com/lancement-du-reseau-etoile/,http://fncdg.com/wp-content/uploads/2017/02/re...,FNCDG,xpath_source,http://fncdg.com/actualites/
8,None,Les décrets n°2017-397 et n°2017-318 du 24 mar...,"<p style=""text-align: justify;"">Les décrets n°...",5 avril 2017,34,fr,None,Le cadre d’emplois des agents de police munici...,http://fncdg.com/le-cadre-demplois-des-agents-...,http://fncdg.com/wp-content/uploads/2017/04/po...,FNCDG,xpath_source,http://fncdg.com/actualites/
9,None,Une candidate à un examen professionnel organi...,"<p style=""text-align: justify;"">Une candidate ...",6 juillet 2017,35,fr,None,Le Conseil d’Etat confirme la souveraineté des...,http://fncdg.com/le-conseil-detat-confirme-la-...,http://fncdg.com/wp-content/uploads/2017/07/Co...,FNCDG,xpath_source,http://fncdg.com/actualites/


In [46]:
# df_articles['art_title'] = df_articles['art_title'].replace(' ', np.nan, inplace=True)
df_articles.dropna(subset=['art_title', 'art_url'])
df_articles['art_title']
df_articles = df_articles.reset_index(drop=True)

df_articles['art_date'] = df_articles['art_extract_datetime'].apply(date_propre)
df_articles['art_date'] = df_articles['art_date'].fillna('24/11/2019')

df_news['art_id'] = df_news['art_id'].astype('str')
df_resume['art_id'] = df_resume['art_id'].astype('str')
df_gamme_ges['art_id'] = df_gamme_ges['art_id'].astype('str')
df_articles['art_id'] = df_articles['art_id'].astype('str')

df_final = df_articles

df_gamme_ges = df_gamme_ges[df_gamme_ges['gamme_gestion']==1]
df_news = df_news[df_news['Innovant']==1]

df_final = df_final.merge(df_news, on='art_id')
df_final = df_final.merge(df_gamme_ges,on='art_id')
df_final = df_final.merge(df_resume,on='art_id')

# ,'art_lang'
df_final = df_final.drop(['Innovant','gamme_gestion','art_content_html','art_extract_datetime','src_type'],axis=1)
list_ind_drop = df_final[df_final['art_title']==' '].index.values.astype(int)
df_final = df_final.drop(list_ind_drop)
df_final = df_final.reset_index(drop=True)
df_final

,art_auth,art_content,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_url,art_date,Nouveau,Theme,art_summary300,art_summary50
0,None,Quels étaient les objectifs poursuivis par le ...,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,http://fncdg.com/actualites/,24/10/2019,0,france,Les quinze mois de concertation avec les organ...,Les quinze mois de concertation avec les organ...
1,None,"La FNCDG mène, en collaboration avec d’autres ...",31,fr,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,http://fncdg.com/wp-content/uploads/2020/12/im...,FNCDG,http://fncdg.com/actualites/,10/12/2020,0,sante,Ce guide a pour vocation de répondre aux quest...,Ce guide a pour vocation de répondre aux quest...
2,None,Le décret n°2018-101 du 16 février 2018 portan...,36,fr,None,Le décret sur la médiation préalable obligatoi...,http://fncdg.com/le-decret-sur-la-mediation-pr...,http://fncdg.com/wp-content/uploads/2018/03/de...,FNCDG,http://fncdg.com/actualites/,13/03/2018,0,sante,"Pour la fonction publique territoriale, cette ...","Pour la fonction publique territoriale, cette ..."
3,None,Cette nouvelle étude présente les données soci...,38,fr,None,Les données sociales nationales 2015 de la Fon...,http://fncdg.com/les-donnees-sociales-national...,http://fncdg.com/wp-content/uploads/2017/10/do...,FNCDG,http://fncdg.com/actualites/,20/10/2017,0,sante,Cette nouvelle étude présente les données soci...,Cette nouvelle étude présente les données soci...
4,None,La loi de finances pour 2017 a été publiée au ...,39,fr,None,Loi n°2016-1917 du 29 décembre 2016 de finance...,http://fncdg.com/loi-n2016-1917-du-29-decembre...,http://fncdg.com/wp-content/uploads/2017/01/lo...,FNCDG,http://fncdg.com/actualites/,11/01/2017,0,sante,L’article 146 de la loi de finances pour 2016 ...,L’article 146 de la loi de finances pour 2016 ...
5,None,La loi n° 2019-828 du 6 août 2019 de transform...,40,fr,None,Loi transformation Fonction Publique,http://fncdg.com/loi-transformation-fonction-p...,http://fncdg.com/wp-content/uploads/2019/08/fu...,FNCDG,http://fncdg.com/actualites/,08/08/2019,0,sante,La loi n° 2019-828 du 6 août 2019 de transform...,La loi n° 2019-828 du 6 août 2019 de transform...
6,None,La loi n°2017-86 du 27 janvier 2017 relative à...,54,fr,None,Promulgation de la loi Égalité et Citoyenneté,http://fncdg.com/promulgation-de-la-loi-egalit...,http://fncdg.com/wp-content/uploads/2017/02/lo...,FNCDG,http://fncdg.com/actualites/,23/02/2017,0,sante,L’article 158 prévoit la réalisation par le Go...,L’article 158 prévoit la réalisation par le Go...
7,None,Le décret n°2014-513 du 20 mai 2014 a créé un ...,65,fr,None,Report de la généralisation du RIFSEEP,http://fncdg.com/report-de-la-generalisation-d...,http://fncdg.com/wp-content/uploads/2017/01/RI...,FNCDG,http://fncdg.com/actualites/,11/01/2017,0,sante,Ils fixent également la liste des corps de l’E...,Ils fixent également la liste des corps de l’E...
8,None,Service à compétence nationale rattaché au dir...,71,fr,None,Signature de la convention entre la FNCDG et l...,http://fncdg.com/signature-de-la-convention-en...,http://fncdg.com/wp-content/uploads/2020/12/DS...,FNCDG,http://fncdg.com/actualites/,16/12/2020,0,sante,Défense mobilité est également au service des ...,Défense mobilité est également au service des ...
9,Citopia,La date du 11 mai est désormais imminente. A l...,77,fr,[Relations Citoyennes],Distribution de masques dans votre commune : c...,https://www.citopia.fr/blog/2020/05/07/distrib...,https://www.citopia.fr/wp-content/uploads/2020...,citopia,https://www.citopia.fr/,07/05/2020,0,international,"Sur votre page Facebook, vous pouvez indiquer ...","Sur votre page Facebook, vous pouvez indiquer ..."


In [47]:
liste_annee=[]
for i in range (len(df_final['art_date'])):
    if int(str.split(df_final['art_date'][i],'/')[2]) not in liste_annee:
        liste_annee.append(int(str.split(df_final['art_date'][i],'/')[2]))
liste_annee.sort()

In [48]:
df_final['art_date'] = pd.to_datetime(df_final['art_date'],format = '%d/%m/%Y')
df_final

,art_auth,art_content,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_url,art_date,Nouveau,Theme,art_summary300,art_summary50
0,None,Quels étaient les objectifs poursuivis par le ...,25,fr,None,"Interview de M. Olivier DUSSOPT, Secretaire d’...",http://fncdg.com/interview-de-m-olivier-dussop...,http://fncdg.com/wp-content/uploads/2019/10/in...,FNCDG,http://fncdg.com/actualites/,2019-10-24,0,france,Les quinze mois de concertation avec les organ...,Les quinze mois de concertation avec les organ...
1,None,"La FNCDG mène, en collaboration avec d’autres ...",31,fr,None,La publication d’un guide de sensibilisation a...,http://fncdg.com/la-publication-dun-guide-de-s...,http://fncdg.com/wp-content/uploads/2020/12/im...,FNCDG,http://fncdg.com/actualites/,2020-12-10,0,sante,Ce guide a pour vocation de répondre aux quest...,Ce guide a pour vocation de répondre aux quest...
2,None,Le décret n°2018-101 du 16 février 2018 portan...,36,fr,None,Le décret sur la médiation préalable obligatoi...,http://fncdg.com/le-decret-sur-la-mediation-pr...,http://fncdg.com/wp-content/uploads/2018/03/de...,FNCDG,http://fncdg.com/actualites/,2018-03-13,0,sante,"Pour la fonction publique territoriale, cette ...","Pour la fonction publique territoriale, cette ..."
3,None,Cette nouvelle étude présente les données soci...,38,fr,None,Les données sociales nationales 2015 de la Fon...,http://fncdg.com/les-donnees-sociales-national...,http://fncdg.com/wp-content/uploads/2017/10/do...,FNCDG,http://fncdg.com/actualites/,2017-10-20,0,sante,Cette nouvelle étude présente les données soci...,Cette nouvelle étude présente les données soci...
4,None,La loi de finances pour 2017 a été publiée au ...,39,fr,None,Loi n°2016-1917 du 29 décembre 2016 de finance...,http://fncdg.com/loi-n2016-1917-du-29-decembre...,http://fncdg.com/wp-content/uploads/2017/01/lo...,FNCDG,http://fncdg.com/actualites/,2017-01-11,0,sante,L’article 146 de la loi de finances pour 2016 ...,L’article 146 de la loi de finances pour 2016 ...
5,None,La loi n° 2019-828 du 6 août 2019 de transform...,40,fr,None,Loi transformation Fonction Publique,http://fncdg.com/loi-transformation-fonction-p...,http://fncdg.com/wp-content/uploads/2019/08/fu...,FNCDG,http://fncdg.com/actualites/,2019-08-08,0,sante,La loi n° 2019-828 du 6 août 2019 de transform...,La loi n° 2019-828 du 6 août 2019 de transform...
6,None,La loi n°2017-86 du 27 janvier 2017 relative à...,54,fr,None,Promulgation de la loi Égalité et Citoyenneté,http://fncdg.com/promulgation-de-la-loi-egalit...,http://fncdg.com/wp-content/uploads/2017/02/lo...,FNCDG,http://fncdg.com/actualites/,2017-02-23,0,sante,L’article 158 prévoit la réalisation par le Go...,L’article 158 prévoit la réalisation par le Go...
7,None,Le décret n°2014-513 du 20 mai 2014 a créé un ...,65,fr,None,Report de la généralisation du RIFSEEP,http://fncdg.com/report-de-la-generalisation-d...,http://fncdg.com/wp-content/uploads/2017/01/RI...,FNCDG,http://fncdg.com/actualites/,2017-01-11,0,sante,Ils fixent également la liste des corps de l’E...,Ils fixent également la liste des corps de l’E...
8,None,Service à compétence nationale rattaché au dir...,71,fr,None,Signature de la convention entre la FNCDG et l...,http://fncdg.com/signature-de-la-convention-en...,http://fncdg.com/wp-content/uploads/2020/12/DS...,FNCDG,http://fncdg.com/actualites/,2020-12-16,0,sante,Défense mobilité est également au service des ...,Défense mobilité est également au service des ...
9,Citopia,La date du 11 mai est désormais imminente. A l...,77,fr,[Relations Citoyennes],Distribution de masques dans votre commune : c...,https://www.citopia.fr/blog/2020/05/07/distrib...,https://www.citopia.fr/wp-content/uploads/2020...,citopia,https://www.citopia.fr/,2020-05-07,0,international,"Sur votre page Facebook, vous pouvez indiquer ...","Sur votre page Facebook, vous pouvez indiquer ..."


In [49]:
df_final[df_final['Nouveau']==1]

,art_auth,art_content,art_id,art_lang,art_tag,art_title,art_url,src_img,src_name,src_url,art_date,Nouveau,Theme,art_summary300,art_summary50
582,Not Found,"par Christoph Demmke ' ""Christoph Demmke est p...",3516,fr,[Not Found],Réforme de la fonction publique en Europe : qu...,https://www.economie.gouv.fr/igpde-editions-pu...,Not Found,Direction générale des Finances publiques,https://www.economie.gouv.fr/dgfip,2019-11-24,1,france,"(2000), The unbearable lightness of skill: the...","(2000), The unbearable lightness of skill: the..."
925,None,M. le président. L'ordre du jour appelle les q...,7123,fr,None,"Déclaration de M. Alain Vidalies, secrétaire d...",https://www.vie-publique.fr/discours/199238-de...,None,Vie publique,www.vie-publique.fr,2016-05-25,1,france,Sauveur Gandolfi-Scheit. Sauveur Gandolfi-Sche...,Sauveur Gandolfi-Scheit. Sauveur Gandolfi-Sche...
974,None,"Mesdames, Messieurs,Chers Amis,Notre réunion, ...",7409,fr,"[International, Relations internationales, Rel...",Déclaration de M. Dominique Galouzeau de Ville...,https://www.vie-publique.fr/discours/141381-de...,None,Vie publique,www.vie-publique.fr,2003-06-13,1,international,Et donc ce travail d'évaluation est un travail...,Et donc ce travail d'évaluation est un travail...
1020,None,Mme la présidente. L'ordre du jour appelle le ...,7702,fr,"[Société, Ville - Territoires, Aménagement ter...","Déclaration de M. Marc Fesneau, ministre charg...",https://www.vie-publique.fr/discours/272248-ma...,None,Vie publique,www.vie-publique.fr,2019-11-19,1,international,"Madame la présidente, monsieur le ministre, me...","Madame la présidente, monsieur le ministre, me..."
1028,None,"ReportageSeillière pour moi, c'est effectiveme...",7763,fr,None,"Interview de M. Ernest-Antoine Seillière, prés...",https://www.vie-publique.fr/discours/205266-in...,None,Vie publique,www.vie-publique.fr,2001-12-09,1,transport,"Philippe de Lyon, par exemple, vous demande Er...","Philippe de Lyon, par exemple, vous demande Er..."
1062,None,"Monsieur le Président,- Messieurs les Présiden...",7928,fr,"[Société, Société - Population, Personne âgée,...","Allocution de M. François Mitterrand, Présiden...",https://www.vie-publique.fr/discours/137052-al...,None,Vie publique,www.vie-publique.fr,1985-02-07,1,international,"Justement, vous avez fait cela pour que le mûr...","Justement, vous avez fait cela pour que le mûr..."
1076,None,"En vérité, comme ce qui s'est entendu ce matin...",8018,fr,None,"Déclaration de M. François Bayrou, président d...",https://www.vie-publique.fr/discours/174175-de...,None,Vie publique,www.vie-publique.fr,2009-02-09,1,transport,"Je crois que c'est une très mauvaise affaire, ...","Je crois que c'est une très mauvaise affaire, ..."
1152,None,"Monsieur le Président,Messieurs les Rapporteur...",8525,fr,"[Institutions, Collectivités territoriales, Ou...","Discours de M. Christian Paul, secrétaire d'Et...",https://www.vie-publique.fr/discours/159065-di...,None,Vie publique,www.vie-publique.fr,2000-11-15,1,international,"Je l'ai dit, le budget traduit dès l'année 200...","Je l'ai dit, le budget traduit dès l'année 200..."
1159,None,1. Le Conseil européen s'est réuni à Bruxelles...,8540,fr,"[Société, Société - Population, Immigration, I...",Conclusions de la présidence du Conseil europé...,https://www.vie-publique.fr/discours/141497-co...,None,Vie publique,www.vie-publique.fr,2003-10-17,1,international,Rappelant les conclusions du Conseil européen ...,Rappelant les conclusions du Conseil européen ...
1257,None,Mme la présidente. L'ordre du jour appelle les...,9182,fr,"[Société, Santé - Protection sociale, Epidémie...","Déclaration de M. Olivier Véran, ministre des ...",https://www.vie-publique.fr/discours/274863-ol...,None,Vie publique,www.vie-publique.fr,2020-05-20,1,international,"Enfin, pouvez-vous garantir, devant la représe...","Enfin, pouvez-vous garantir, devant la représe..."


In [50]:
# def nettoyage(themes_extrait):
#     #Création de la liste de sortie
#     themes_nettoyes=[]
#     #Pour chaque champ, on enlève les caractères 'parasite', on ajoute le résultat dans la liste de sortie
#     for i in range(0,len(themes_extrait)):
#         if ',' in themes_extrait[i]:
#             for j in themes_extrait[i].split(','):
#                 themes_nettoyes.append(j.strip(" '\"[]#"))
#         else:
#             themes_nettoyes.append(themes_extrait[i].strip(" '\"[]#"))
#     #On enlève les doublons
#     themes_nettoyes=list(np.unique(themes_nettoyes))
#     themes_nettoyes.remove('')
#     return themes_nettoyes

def traitement_themes():
    #Import des la liste des thèmes
    themes=list(df_final['art_tag'].dropna())
    #On enlève les doublons
#     #Appel de la fonction extraction
    themes_extrait=[y.strip(" '\"[]#") for x in themes for y in x]
    themes_uniques=list(np.unique(themes_extrait))
    #Appel de la fonction nettoyage
#     themes_finaux=nettoyage(themes)
    #Retourne les thèmes
    return themes_uniques

def Importation(name: str):
    '''Documentation
    Parameters: name refert to the file name
    output: DataFrame
    '''
    return pd.read_excel(name)

#Function to create language options for the dropdown
def get_lang_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_lang: 'lang' field with no errors and duplicates
        dict: dictionary with the languages found in the database
        
    output:
        lang_options: list of dictionaries for dropdowns
    """
    list_lang=articles['art_lang'].dropna().unique()
    #Declaration of lang_options. It needs to be declared in order to use the append method.
    lang_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_lang)):
        dict={'label':list_lang[n],'value':list_lang[n]}
        lang_options.append(dict)
    return lang_options

#Fonction qui estime le temps de lecture
def count_words(string:str):
    """ 
    Parameters:
        string: We take the words of the article as input
    Attributes:
        string1: it allows to remove the beginning and end space
        count: we initialize count to 1, count will be our word counter
        second: we know that we read 275 words in 1 minute, so we calculate the number of seconds it would take to read the article.
        minute: we convert the number of seconds into minutes
        second2: we make the difference between a minute and the whole part of a minute, which is multiplied by 60. This gives us the remaining seconds
    Out:
        out1: maximum estimated reading time in minutes

    """
    string1=str(string).strip()
    count=1
    for i in string1:
        # a for is used to count the number of words in the article
        if i==" ":
            count+=1
    second2=(count*60)/275
    minute=second/60
    second2=(minute-math.floor(minute))*60
    if second2>0:
        return ("temps maximum de lecture estime a",math.floor(minute)+1," min ")
    else:
        return ("temps maximum de lecture estime a",math.floor(minute)," min ")

#Function to create themes options for the dropdown
def get_cat_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_cat: 'themes' field
        dict: dictionary with the languages found in the database
        
    output:
        cat_options: list of dictionaries for dropdowns
    """
    list_cat=articles['src_name'].dropna().unique()
    #Declaration of cat_options. It needs to be declared in order to use the append method.
    cat_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_cat)):
        dict={'label':list_cat[n],'value':list_cat[n]}
        cat_options.append(dict)
    return cat_options


def get_min_max_dates(articles):
    """Documentation
    
    Parameters:
        articles: we take dataframe type articles as inputs
        
    Attributes:
        date: we retrieve the column art_date where the dates are stored

    Out:
        min_date: the minimum date of the column is retrieved
        max_date: the maximum date of the column is retrieved
    """
    date = articles[['art_date']]
    min_date = min(date['art_date'])
    max_date = max(date['art_date'])
    return (min_date,max_date)

def plot_pie_chart(articles, variable : str , pie_title : str): 
    """Documentation
    
    Parameters:
    
    articles : DataFrame used to plot
    variable : Variable used to compare articles
    pie_title: Title given to the plot
    
    Out: figure 
    """
    articles[variable].fillna("Unknown", inplace = True)
    grp_variable = articles.groupby(variable)[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig = px.pie(grp_variable, values='nb_articles', names=variable, title=pie_title)
    return fig


#graphs
# df['art_lang'].fillna("Unknown", inplace = True)
# grp_languages = df.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
# fig_lang = plot_pie_chart(df,'art_lang','Languages')


# df2=df.groupby('src_name')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
# fig_themes = px.histogram(df2, x="src_name", y='nb_articles')


In [62]:
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

df = df_final.reset_index(drop=True)


#recuperer la liste des themes
themes = df["Theme"].unique()
liste_theme=[]
for theme in themes:
    if theme not in liste_theme:
        liste_theme.append(theme)

tags = traitement_themes()
dic = {}
for i in df['art_tag']:
    if i is None :
        continue
    else :
        for j in tags:
            if j in i:
                if j in dic:
                    dic[j] += 1
                else :
                    dic[j] = 1
                    
top_tags = sorted(dic.items(),key = lambda t:t[1])[-11:-1]

tags_index = 11

# Search Bar

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(id = "username",type="search", placeholder="Search",debounce=True)),
        dbc.Col(
            dbc.Button("Search", id = "submit-button",color="primary", className="ml-2",href="/"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Nav Bar

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Berger-levrault", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://www.berger-levrault.com/fr/",
        ),
        dbc.NavLink("Article", active=True, href="/"),
        dbc.NavLink("Statistiques", href="/stats"),
        dbc.NavLink("Stories",href="/stories"),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
    fixed = "top",
    sticky = "top",
)


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "margin-left": "2rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

#Side bar

sidebar = html.Div(
    [
        html.H2("Home"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Number of article par page'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10,
                    min = 1,
                    max = len(df['art_title']),
                    )])),
            dbc.Col(html.Div([html.Label('Themes'),
                              dcc.Dropdown(id='cat_filt',
                    options=[
                                        {"label": row, "value": row} for row in liste_theme
                                    ],
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date= min(df['art_date']),
                    end_date = datetime.date.today(),
                    display_format='DD/MM/YYYY'
                    # end_date_placeholder_text='Select a date!'
                    )]), width='auto'),
            html.Hr(),
            dbc.Col(html.Div([
                              html.Button('Start research',
                                          id='research', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})])
                   ),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)

##Side_bar_stats

sidebar_stats = html.Div(
    [
        html.H2("Stats"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([dbc.Label("Année"),
                                dcc.Dropdown(
                                    id="annee-var",
                                    options=[
                                        {"label": row, "value": row} for row in liste_annee
                                    ],
                                    value="2020",
                                ),
                             ])
                   ),
            dbc.Col(html.Div([dbc.Label("Thème"),
                                dcc.Dropdown(
                                    id="theme-var",
                                    options=[
                                        {"label": row, "value": row} for row in liste_theme
                                    ],
                                    value="france",
                                ),]), width='auto'),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)


#Nav bar for pagging

nb_pages = 10

pages = dbc.NavbarSimple(
    children=[
        dbc.Col(html.Div(html.H5("Page", style = {"color":"white"}))),
        dbc.Col(html.Div([dcc.Input(
                    id = 'num_page',
                    type= "number",
                    value = 1,
                    min = 1,
                    style = {"border-radius": "8px", "color": "black", "border": "2px solid #e7e7e7",
                            "width" : "30%",
                            "background-color": "#eee"}
                    )])),
    ],
    color="dark",
)

#Stories

def render_stories():
    list_tags = []
    for i in range(len(top_tags)-1,-1,-1):
        list_tags.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([
                    dbc.CardBody(
                        [
                            html.H5(top_tags[i][0], className="card-title"),
                            html.H6("Number of article : " + str(top_tags[i][1]), className="card-subtitle"),
                            dbc.Button("Show this tags",id = {'type': 'tags-button','index': i}),   
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    
    
    return list_tags 



# Content page

card = html.Div(id ='out_card')

article = html.Div(id ='out_article')

stories = html.Div(render_stories())


app.layout = html.Div(
        [
            navbar,
            html.Hr(),
            dcc.Location(id='url', refresh=False),
            html.Div(id='page-content'),
            pages
        ]
)

index_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(card),
                                dbc.Col(stories,width = "30%")
                            ],
                            no_gutters=True,
                        ),

stats_page_layout = dbc.Row([
                                dbc.Col([sidebar_stats], width = "100%"),
                                dbc.Col(html.Div(
                                            dcc.Graph(id="cluster-graph"),
                                       )
                                )
                            ],
                    no_gutters=True
        )

article_page_layout = dbc.Col(article)

stories_page_layout = dbc.Col(stories)


# Callback

# add callback for toggling the collapse on small screens
@app.callback(
    dash.dependencies.Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


@app.callback(
    dash.dependencies.Output('out_card', 'children'),
    dash.dependencies.Output('num_page','max'),
    
    [Input('research','n_clicks')],
    [Input('num_page','value')],
    [Input({'type': 'tags-button','index': ALL},'n_clicks')],
    [Input('submit-button', 'n_clicks')],
    [Input('username', 'value')],
    
    [State('cat_filt','value')],
    [State('number', 'value')],
    [State('date-picker-range','start_date')],
    [State('date-picker-range','end_date')],
    [State('username', 'value')]
)

def update_card_div(n_1,num_page,n_2,clicks,event,theme,number,start_date,end_date,input_value):
    
    global tags_index
    start_date = start_date[:10]
    
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    filtered_df = df[df['Nouveau']==1]
    
    if n_1 is not None:
        filtered_df = df
        if theme is not None:
            filtered_df = filtered_df[filtered_df['Theme'].isin(theme)]

        filtered_df = filtered_df.reset_index(drop=True)
    
        if start_date is not None and end_date is not None:
            filtered_df = filtered_df[(filtered_df['art_date'] >= start_date) & (filtered_df['art_date'] <= end_date)]

        filtered_df = filtered_df.reset_index(drop=True)
    
    if n_2 is not None:
        filtered_df = df
        
        filtered_df['art_tag'] = filtered_df['art_tag'].astype('str')
        if "index" in changed_id:
            tags_index = int(changed_id[9])
            filtered_df = filtered_df[filtered_df['art_tag'].str.contains(top_tags[tags_index][0],case = False)]
        elif tags_index != 11:
            filtered_df = filtered_df[filtered_df['art_tag'].str.contains(top_tags[tags_index][0],case = False)]

        filtered_df = filtered_df.reset_index(drop=True)
    
    element = []
    if clicks is not None or event is not None:
        filtered_df = df
        for nb in range(len(filtered_df)):
            t = input_value
            if t.lower() in filtered_df['art_title'][nb].lower() or t.lower() in filtered_df['art_content'][nb].lower() or t.lower() in filtered_df['art_tag'][nb].lower():
                element.append(filtered_df['art_id'][nb])
                
        if element != []:
            filtered_df = filtered_df[filtered_df['art_id'].isin(element)]
 
        filtered_df = filtered_df.reset_index(drop=True)
    
    length_df = len(filtered_df['art_title'])
    
    if num_page*number > length_df:
        end = length_df
        filtered_df = filtered_df[((num_page*number)-number):length_df]
    else :
        filtered_df = filtered_df[((num_page*number)-number):((num_page*number))]
        end = num_page*number
    
    cards = []
    
    if (tags_index != 11):
        cards = [html.H5(top_tags[tags_index][0]),html.Hr()]
    
    for i in range(((num_page*number)-number),end):
        cards.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=filtered_df['src_img'][i],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(filtered_df['art_title'][i], className="card-title"), href='/page-'+str(filtered_df['art_id'][i])),
                            html.P(filtered_df['art_summary50'][i])
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    return cards,(length_df//number)+1

#Content article

@app.callback(
    Output('cat_filt', 'value'),
    Output('date-picker-range', 'start_date'),
    Output('date-picker-range', 'end_date'),
    Input('reset_button','n_clicks')
)

def reset_filter(n_click):
    global tags_index
    
    if n_click is None:
        raise PreventUpdate
    
    tags_index = 11
    return [],min(df['art_date']),datetime.date.today()

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
    
    index = int(pathname.split("-")[1])
    
    df_article = df[df['art_id'] == index]
    df_article = df_article.reset_index()

    article = [html.Div([
            dbc.Card([
                dbc.CardImg(src=df_article['src_img'][0],alt="Can't display this image",top=True),
                dbc.CardBody(
                    [
                        dbc.CardLink(html.H5(df_article['art_title'][0], className="card-title"), href=df_article['art_url'][0]),
                        html.P(df_article['art_summary300'][0])
                    ]
                )],
                className="w-100 mb-3"
            ),
        ])
    ]

    return article


# Callback for update data in graphs
@app.callback(
    Output("cluster-graph", "figure"),
    [
        Input("annee-var", "value"),
        Input("theme-var", "value"),
    ],
)
def nb_articles(annee,theme):
    l = [0]*12
    articles_2 = df
    
    
    articles_2 = articles_2[articles_2['Theme']==theme]
    
    articles_2 = articles_2.reset_index(drop=True)
    
    for i in range(1,len(articles_2['art_date'])):
        mois=int(articles_2['art_date'][i].strftime("%m"))
        annee2=int(articles_2['art_date'][i].strftime("%Y"))
        if int(annee2)==int(annee):
            l[mois-1]+=1
    
    fig = go.Figure(
    data=[go.Scatter(x=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'], y=l)],
    layout=go.Layout(
        title=go.layout.Title(text="Nombre d'article publié par mois")
                             
    )
)

    return (fig)

#Content    
    
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return index_page_layout
    elif pathname == "/stats":
        return stats_page_layout
    elif pathname == "/stories":
        return stories_page_layout
    else:
        return article_page_layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )




if __name__ == "__main__":
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:8050/
